Pobieranie wiadomości z Onetu
Zapis do pliku ./data/onet_{timestamp}.txt

Na początek instalacja brakujących pakietów (opcjonalne):

In [1]:
# ! pip install bs4
# ! pip install requests
# ! pip install tqdm
# ! pip install pandas
# ! pip install pyarrow
# ! pip install qgrid
# ! pip install blaze
# ! pip install ipynbname

In [2]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm, trange
from datetime import datetime, timezone
from common.webarticle import WebArticle
from common.pl_helper import PolishLocaleHelper
import pandas as pd
import logging
import ipynbname

In [3]:
# settings:
portal_name = 'onet'
website = 'https://wiadomosci.onet.pl/kraj'
pagination_attributes = '?ajax=1&page='
no_articles_to_scrap = 300
data_dir = '../data'
logging.basicConfig(
    filename=f'../logs/{ipynbname.name()}.ipynb.log',
    encoding='utf-8',
    level=logging.DEBUG
)
ignored_paragraph_starts = ['Zobacz też:']

Sprawdźmy plik robots.txt, w czasie pisania tego kodu pozwalał na pobieranie plików z onetu bez specjalnych ograniczeń

In [4]:
resp = requests.get('https://www.onet.pl/robots.txt')
print(resp.text)

User-Agent: *
Disallow: 
Allow: /


In [5]:
def check_if_ignored_paragraph(text, ignored_paragraphs, operation='startswith'):

    for ignored_paragraph in ignored_paragraphs:
        if operation == 'startswith':
            if text.startswith(ignored_paragraph):
                return True
        else:
            logging.debug(f'check_if_ignored: operation {operation} not implemented')
    return False


In [6]:
def scrap_article_text_onet(link):
    logging.info('scrap_article_text_onet:')
    logging.info(f'article link to scrap: {link}')

    page = requests.get(link)
    if page.status_code != 200:
        logging.debug(f'received status code: {page.status_code}')
        return '', ''

    soup = BeautifulSoup(page.content, 'html.parser')
    paragraphs = soup.find_all('p', class_='hyphenate')

    logging.info(f'paragraphs found: {paragraphs}')

    text = ''
    for paragraph in paragraphs:
        paragraph_text = paragraph.get_text().strip()
        if check_if_ignored_paragraph(paragraph_text, ignored_paragraph_starts):
            continue
        text += paragraph_text + ' '

    time_string = soup.find('span', class_='datePublishedContent').get_text().strip()
    when_published = PolishLocaleHelper.parse_pl_date_onet(time_string)
    title_int = soup.find('h1', class_='mainTitle').get_text().strip()
    lead_text_int = soup.find('div', class_='hyphenate lead').get_text().strip()
    author = None
    author_candidate = soup.find('div', class_='authorSourceProfile authorProfile')
    if author_candidate is not None:
        author = author_candidate.find('span', class_='name').get_text(strip=True)

    source = None
    source_candidate = soup.find('span', itemprop='sourceOrganization')
    if source_candidate is not None:
        source = source_candidate.get_text().strip()

    logging.info(f'text: {text}')
    logging.info(f'when published: {when_published}')
    logging.info(f'title_int: {title_int}')
    logging.info(f'lead_text_int: {lead_text_int}')
    logging.info(f'author: {author}')
    logging.info(f'source: {source}')

    return text, when_published, title_int, lead_text_int, author, source

In [7]:
def scrap_articles_onet(website):
    logging.info('scrap_articles_onet:')
    logging.info(f'page with articles to scrap: {website}')

    result = list()
    page = requests.get(website)
    if page.status_code != 200:
        logging.debug(f'received status code: {page.status_code}')
        return result

    soup = BeautifulSoup(page.content, 'html.parser')
    page_content = soup.find_all('div',class_='listItem listItemSolr itarticle')

    for article in page_content:
        title = article.find(class_='itemTitle').get_text().strip()
        lead_text = article.find(class_='itemLead hyphenate').get_text().strip()
        link = article.find_all('a', href=True)[2]['href']
        webart = WebArticle(title, lead_text, link, *scrap_article_text_onet(link),portal_name)
        logging.info(f'webarticle scrapped: {webart}')
        result.append(webart)

    return result

In [8]:
no_scrapped_articles = 0
page_no=0
scrapped_articles = list()

logging.info(f'no_articles_to_scrap: {no_articles_to_scrap}')

pbar = tqdm(total=no_articles_to_scrap)
while no_scrapped_articles < no_articles_to_scrap:
    logging.info('next page: '+website+pagination_attributes+str(page_no))
    new_articles = scrap_articles_onet(website+pagination_attributes+str(page_no))
    page_size = len(new_articles)
    logging.info(f'new_articles: {page_size}')
    if no_scrapped_articles + page_size > no_articles_to_scrap:
        page_size = no_articles_to_scrap - no_scrapped_articles
    scrapped_articles.extend(new_articles[:page_size])
    logging.info(f'scrapped new articles: {page_size}')
    no_scrapped_articles += page_size
    pbar.update(page_size)
    logging.info(f'no_scrapped_articles (total): {no_scrapped_articles}')
    page_no += 1
    if len(scrapped_articles) <= 0:
        logging.debug(f'0 articles scrapped!')
        break
pbar.close()

100%|██████████| 300/300 [03:19<00:00,  1.51it/s]


In [9]:
for i in trange(len(scrapped_articles)):
    # print(f'{i}: {scrapped_articles[i].lead_text}')
    print(f'{i}: {len(scrapped_articles[i].text)}')

100%|██████████| 300/300 [00:00<00:00, 13053.08it/s]

0: 2024
1: 2200
2: 2206
3: 5173
4: 2517
5: 4333
6: 2810
7: 2533
8: 2851
9: 3287
10: 3883
11: 2975
12: 2256
13: 4983
14: 3370
15: 2464
16: 2629
17: 5203
18: 7303
19: 3379
20: 1952
21: 1158
22: 0
23: 1834
24: 1737
25: 3416
26: 1706
27: 2262
28: 3665
29: 0
30: 0
31: 0
32: 0
33: 3039
34: 1954
35: 2232
36: 0
37: 0
38: 1915
39: 4165
40: 1421
41: 3615
42: 1514
43: 2075
44: 3194
45: 2098
46: 3616
47: 658
48: 3298
49: 1945
50: 2787
51: 2131
52: 2076
53: 3072
54: 1322
55: 1635
56: 4698
57: 5042
58: 2170
59: 2691
60: 7486
61: 3003
62: 4031
63: 3442
64: 2619
65: 2806
66: 2905
67: 2178
68: 5294
69: 3053
70: 0
71: 2925
72: 1876
73: 0
74: 5478
75: 1504
76: 3257
77: 1616
78: 1068
79: 4772
80: 537
81: 3649
82: 2829
83: 2274
84: 542
85: 1130
86: 2951
87: 1801
88: 3202
89: 38
90: 2847
91: 3977
92: 4004
93: 2779
94: 3597
95: 1573
96: 5367
97: 1396
98: 1572
99: 967
100: 0
101: 1007
102: 2562
103: 1589
104: 2496
105: 868
106: 1219
107: 5546
108: 2646
109: 4632
110: 2915
111: 1916
112: 730
113: 1362
114: 137

In [10]:
onet_dataframe = pd.DataFrame(article.__dict__ for article in scrapped_articles)
onet_dataframe

,title,title_int,lead_text,lead_text_int,link,text,when_published,author,source,portal
0,Adam Niedzielski: mamy eksplozję pandemii,Adam Niedzielski: mamy eksplozję pandemii,Ostatnie dwa dni możemy nazwać eksplozją pande...,Ostatnie dwa dni możemy nazwać eksplozją pande...,https://wiadomosci.onet.pl/kraj/koronawirus-ad...,Pytany o scenariusze rozwoju pandemii w Polsce...,2022-01-19 20:08:00,None,Media,onet
1,Mark Brzeziński nowym ambasadorem USA w Polsce...,Mark Brzeziński nowym ambasadorem USA w Polsce...,Mark Brzeziński został zaprzysiężony na nowego...,Mark Brzeziński został zaprzysiężony na nowego...,https://wiadomosci.onet.pl/kraj/mark-brzezinsk...,W styczniu z Polską pożegnała się Georgette Mo...,2022-01-19 20:05:00,None,Media,onet
2,Minister spraw zagranicznych zakażony koronawi...,Minister spraw zagranicznych zakażony koronawi...,Informację o pozytywnym wyniku testu na SARS-C...,Informację o pozytywnym wyniku testu na SARS-C...,https://wiadomosci.onet.pl/kraj/minister-spraw...,"""Z uwagi na pozytywny wynik testu na SARS-CoV-...",2022-01-19 19:49:00,None,Media,onet
3,Komisja do spraw podsłuchów coraz bliżej. Kuki...,Komisja do spraw podsłuchów coraz bliżej. Kuki...,"— Nie chcę dopuścić do takiej sytuacji, by alb...","— Nie chcę dopuścić do takiej sytuacji, by alb...",https://wiadomosci.onet.pl/kraj/komisja-ds-pod...,"W ubiegłym tygodniu Paweł Kukiz poinformował, ...",2022-01-19 19:35:00,None,PAP,onet
4,Prezydent złożył poprawki do Polskiego Ładu,Prezydent złożył poprawki do Polskiego Ładu,Kancelaria Prezydenta przekazała przedstawicie...,Kancelaria Prezydenta przekazała przedstawicie...,https://wiadomosci.onet.pl/kraj/prezydent-zloz...,W środę odbyły się konsultacje kierownictwa Ka...,2022-01-19 18:59:00,None,PAP,onet
...,...,...,...,...,...,...,...,...,...,...
295,Warszawska konferencja o przyszłości Europy. U...,Warszawska konferencja o przyszłości Europy. U...,Przez trzy dni 119 uczestników z całej Unii Eu...,Przez trzy dni 119 uczestników z całej Unii Eu...,https://wiadomosci.onet.pl/kraj/konferencja-w-...,Konferencja o przyszłości Europy to szeroko za...,2022-01-10 13:20:00,Piotr Maciej Kaczyński,Onet,onet
296,Internauci bezlitośni dla Polskiego Ładu. Zoba...,Internauci bezlitośni dla Polskiego Ładu. Zoba...,Bałagan w przepisach i zamieszanie wśród podat...,Bałagan w przepisach i zamieszanie wśród podat...,https://wiadomosci.onet.pl/kraj/polski-lad-zob...,Żarty z Polskiego Ładu pojawiły się w sieci ju...,2022-01-10 12:51:00,None,None,onet
297,Donald Tusk oddał telefon ekspertom. Nie stwie...,Donald Tusk oddał telefon ekspertom. Nie stwie...,Na telefonie przewodniczącego PO nie było używ...,Na telefonie przewodniczącego PO nie było używ...,https://wiadomosci.onet.pl/kraj/tusk-oddal-tel...,"Jak dodaje Grabiec, jeśli Tusk był inwigilowan...",2022-01-10 12:27:00,None,Media,onet
298,Kolejny polski dyplomata skrytykował rząd i st...,Kolejny polski dyplomata skrytykował rząd i st...,W poniedziałek rzecznik Ministerstwa Spraw Zag...,W poniedziałek rzecznik Ministerstwa Spraw Zag...,https://wiadomosci.onet.pl/kraj/kolejny-polski...,Jarosław Nowak objął stanowisko pełnomocnika r...,2022-01-10 12:10:00,None,Media,onet


In [11]:
timestamp = int((datetime.now(timezone.utc)).timestamp())

In [12]:
onet_dataframe.to_parquet(f'{data_dir}/{portal_name}_{timestamp}.parquet')
onet_dataframe.to_json(f'{data_dir}/{portal_name}_{timestamp}.json')

Krótki test - dane pierwszego artykułu

In [13]:
print(f'title:\n  {onet_dataframe.iloc[0]["title"]}')
print(f'lead_text:\n  {onet_dataframe.iloc[0]["lead_text"]}')
print(f'link:\n  {onet_dataframe.iloc[0]["link"]}')
print(f'text:\n  {onet_dataframe.iloc[0]["text"]}')
print(f'when_published:\n  {onet_dataframe.iloc[0]["when_published"]}')

title:
  Adam Niedzielski: mamy eksplozję pandemii
lead_text:
  Ostatnie dwa dni możemy nazwać eksplozją pandemii — powiedział Adam Niedzielski na antenie TVP Info. Przypomniał, że przez całą jesień nie było wyniku powyżej 30 tys., a teraz zaczynamy V falę z takiego pułapu.
link:
  https://wiadomosci.onet.pl/kraj/koronawirus-adam-niedzielski-mamy-eksplozje-pandemii/l8rhyqc
text:
  Pytany o scenariusze rozwoju pandemii w Polsce powiedział: "Realizuje się scenariusz bardzo szybkiego przyspieszenia. Na początku lutego możemy przekroczyć 100 tys. Ale ten scenariusz zakłada, że liczba zakażonych może szybko spadać". Niedzielski był pytany, czy może się w Polsce spełnić scenariusz, w którym będzie np. 500 tys. zakażeń. — Kluczowe, w jakim stopniu zakażenia będą przekładać się na hospitalizację. Jak obserwujemy w Europie widać, że wyraźnie jest mniej hospitalizacji. Ale zwracam uwagę, że wnioskujemy to, patrząc na przypadek Wielkiej Brytanii i Francji, ale tam jest wyższy współczynnik wyszcze

In [14]:
print(f'title:\n  {onet_dataframe.iloc[21]["title"]}')
print(f'lead_text:\n  {onet_dataframe.iloc[21]["lead_text"]}')
print(f'link:\n  {onet_dataframe.iloc[21]["link"]}')
print(f'text:\n  {onet_dataframe.iloc[21]["text"]}')
print(f'when_published:\n  {onet_dataframe.iloc[21]["when_published"]}')

title:
  DJ Antex nie żyje. Artysta przegrał walkę z koronawirusem
lead_text:
  Damian Antoniewski, szerzej znany jako DJ ANTEX nie żyje. Artysta w ostatnich tygodniach ciężko przechodził zakażenie koronawirusem. Zmarł w szpitalu w Oleśnie na Opolszczyźnie w wieku 42-lat.
link:
  https://wiadomosci.onet.pl/kraj/dj-antex-nie-zyje-artysta-w-czasie-swiat-zarazil-sie-koronawirusem/p17bgle
text:
  DJ Antex był znanym artystą związanym ze sceną klubową od ponad 20 lat. Występował w całej Polsce, Anglii oraz wielu innych krajach w Europie. Jak informuje TVN24, odszedł w szpitalu w Oleśnie na Opolszczyźnie. 42-letni artysta w Wigilię Bożego Narodzenia poinformował fanów, że jest zakażony koronawirusem. Kilkanaście dni wcześniej zagrał swoją ostatnią imprezę w Legnicy. "Serdeczni przyjaciele. Z okazji nadchodzących Świąt Bożego Narodzenia chciałem Wam życzyć wszystkiego najlepszego. Moje święta w tym roku są nieco inne od tych z poprzednich lat, niestety będę musiał je spędzić w samotności bez 